In [1]:
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sktime.utils.load_data import load_from_tsfile_to_dataframe
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F

# from tslearn.neighbors import KNeighborsTimeSeriesClassifier

import logging
logging.basicConfig(filename="1_nn_eucledian_results_history.log", level=logging.INFO)

import warnings
warnings.filterwarnings("ignore")

In [2]:
def simple_preproc_before_knn(X):
    X = X.applymap(np.array)
    dims_lst = X.columns

    for dim_name in dims_lst:
        dim_values = np.stack(X[dim_name].values, axis=0)
        for i in range(dim_values.shape[1]):
            X[dim_name + f'_{i}'] = dim_values[:, i]

    X = X.drop(columns=dims_lst)
    return X


def preproc_answers(y):
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    return y

# Run on all datasets

In [3]:
datasets_directory = "/root/data/Multivariate_ts/"
datasets_names_lst = os.listdir(datasets_directory)

for dataset_name in tqdm(datasets_names_lst[4:]): # start from the 3rd dataset
    logging.info(f'{dataset_name}')
    print(dataset_name)
    
    try:
        X_train, y_train = load_from_tsfile_to_dataframe(datasets_directory\
                                                         + dataset_name + f'/{dataset_name}_TRAIN.ts')
        X_test, y_test = load_from_tsfile_to_dataframe(datasets_directory\
                                                       + dataset_name + f'/{dataset_name}_TEST.ts')
        print('data has been loaded')

        X_train = simple_preproc_before_knn(X_train)
        X_test = simple_preproc_before_knn(X_test)

        y_train = preproc_answers(y_train)
        y_test = preproc_answers(y_test)
        print('data has been preprocessed')

#         knn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
        knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
        knn.fit(X_train, y_train)

        test_predictions = knn.predict(X_test)
        test_accuracy = accuracy_score(test_predictions, y_test)
        logging.info(f"test_accuracy: {test_accuracy}\n")
        print('test_accuracy:',test_accuracy)
        print('\n')
    except:
        print('Error: smth is wrong\n')
        logging.info(f"Error: smth is wrong\n")

CharacterTrajectories
data has been loaded
Error: smth is wrong




# One dataset example

In [30]:
# def preproc_before_knn_dtw(X):
#     X = X.applymap(np.array)

#     dimensions_lst = []

#     for dim in X.columns:
#         dimensions_lst.append(np.dstack(list(X[dim].values))[0])

#     dimensions_lst = np.array(dimensions_lst)
#     X = torch.from_numpy(np.array(dimensions_lst, dtype=np.float64))
#     X = X.transpose(0, 2)
#     X = X.transpose(1, 2)
#     X = F.normalize(X, dim=1)
#     X = X.float().numpy()
#     return X

In [6]:
datasets_directory = "/root/data/Multivariate_ts/"

# dataset_name = 'ERing'
# dataset_name = 'InsectWingbeat'
# dataset_name = 'Cricket'
# dataset_name = 'CharacterTrajectories'
dataset_name = 'LSST'


X_train, y_train = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TRAIN.ts')
X_test, y_test = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TEST.ts')

In [7]:
ms = preprocessing.MinMaxScaler()

X_train = simple_preproc_before_knn(X_train)
X_test = simple_preproc_before_knn(X_test)

X_train = ms.fit_transform(X_train)
X_test = ms.transform(X_test)

y_train = preproc_answers(y_train)
y_test = preproc_answers(y_test)

# print(X_train.shape)
# print(X_test.shape)

In [8]:
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
# knn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train, y_train)

train_predictions = knn.predict(X_train)
train_accuracy = accuracy_score(train_predictions, y_train)
print('train_accuracy:', round(train_accuracy,5))

test_predictions = knn.predict(X_test)
test_accuracy = accuracy_score(test_predictions, y_test)
print('test_accuracy:', round(test_accuracy,5))

train_accuracy: 1.0
test_accuracy: 0.41403
